In [153]:
import pandas as pd
import numpy as np
import os
from itertools import product
import pdb
import matplotlib.pyplot as plt

In [163]:
filename_='prolific_dat.csv'

data_sf = pd.read_csv(filename_, header = 0)
data_sf = data_sf.rename(columns={'Subject ID': 'Subject'})
data_sf.Subject =data_sf.Subject.apply(lambda x : x[:-1])
print(data_sf.columns)
print(data_sf.index)

filename_='prolific_dat_longfirst.csv'

data_lf = pd.read_csv(filename_, header = 0)
data_lf = data_lf.rename(columns={'Subject ID': 'Subject'})
data_lf.Subject = data_lf.Subject.apply(lambda x : x[:-1])
print(data_lf.columns)
print(data_lf.index)

Index(['Subject', 'Block', 'TreeNumber', 'TimeRemaining', 'TimeStamp',
       'Action', 'TravelTime', 'RequiredPresses', 'CurrentPresses',
       'KeyPressIncreaseRate', 'score'],
      dtype='object')
RangeIndex(start=0, stop=68947, step=1)
Index(['Subject', 'Block', 'TreeNumber', 'TimeRemaining', 'TimeStamp',
       'Action', 'TravelTime', 'RequiredPresses', 'CurrentPresses',
       'KeyPressIncreaseRate', 'score'],
      dtype='object')
RangeIndex(start=0, stop=71494, step=1)


The data collection for pilot data is messed up 

Needs to be fixed before proceeding

In [164]:
data_sf['TreeNumberCorrected'] = np.nan

treenum = 1

for i, ser in data_sf.iterrows():
    # print(i)
    if i == 0:
        data_sf.loc[i, 'TreeNumberCorrected'] = 1
        continue

    if data_sf.loc[i, 'Block'] != data_sf.loc[i-1, 'Block']:
        data_sf.loc[i, 'TreeNumberCorrected'] = 1

    elif data_sf.loc[i, 'RequiredPresses'] >= data_sf.loc[i-1, 'RequiredPresses']:
        data_sf.loc[i, 'TreeNumberCorrected'] = data_sf.loc[i-1, 'TreeNumberCorrected']
    elif data_sf.loc[i, 'RequiredPresses'] < data_sf.loc[i-1, 'RequiredPresses']:

        data_sf.loc[i, 'TreeNumberCorrected'] = data_sf.loc[i-1, 'TreeNumberCorrected'] + 1

data_sf['TreeNumber'] = data_sf['TreeNumberCorrected']
data_sf = data_sf.astype({"TreeNumber":int})
data_sf.to_csv('prolific_dat_corrected.csv', index = False, header = True)


In [165]:
def resetTreeNum(data):
    try:
        data['TreeNumber'] = data.groupby(["Subject", "Block"], group_keys=False)[['TreeNumber']].apply(lambda x: x - np.min(x))
    except:
        print("TreeNumber wasn't in the column list");
    return data

In [166]:
data_lf = resetTreeNum(data_lf)

In [172]:
def create_condensed_df(data, shortFirst = True):

    subjects = data['Subject'].unique()
    blocks = [0, 1]
    headers =  ['subject', 'env', 'tree', 'tree_type', 'harvest_dur', 'num_harvests', 'difficulty_at_exit', 'score_tree', 
                'dur_per_harvest', 'num_presses']
    har_dur_df_list = []
    for subj, block in product(subjects, blocks):
        print(f"Block : {block}, Subj: {subj}")
        subj_df = data[(data['Subject']==subj) & (data['Block'] == block)]
        trees = np.unique(subj_df['TreeNumber'])
        print(f"trees: {trees}")
        num_trees = np.max(trees)
        harvest_dur = np.zeros((1,np.max(trees)))
        prev_score = 0
        for tree in trees:
            dur_per_harvest = []
            num_presses_arr = []
            scores_harvest = []
            entry_row_idx = subj_df[subj_df['TreeNumber']==tree].index[0]
            exit_row_idx = subj_df[subj_df['TreeNumber']==tree].index[-1]
            
            # check if exit row action is either 'LEAVE' or is the final tree 
            cond_leave = subj_df.loc[exit_row_idx, 'Action'] == 'LEAVE'
            cond_last_tree = subj_df.loc[exit_row_idx, 'TreeNumber'] == num_trees # final tree
    
            # assert cond_leave or cond_last_tree, 'Error: exit row indexing is wrong!'
            # the above condition doesn't work for the pilot as the eventlistener was not removed during the travel space, 
            # so some spurious "PRESS" events were recorded
            
            timestamp_leave = subj_df.loc[exit_row_idx,'TimeStamp']
            timestamp_leave = timestamp_leave.squeeze()
            timestamp_enter = subj_df.loc[entry_row_idx,'TimeStamp']
            
            har_dur = timestamp_leave - timestamp_enter
    
            if subj_df.loc[entry_row_idx,'KeyPressIncreaseRate']==2:
                tree_type = 'good'
            else:
                tree_type = 'bad'
    
            difficulty_at_exit = subj_df.loc[exit_row_idx, 'RequiredPresses']
            difficulty_at_exit = difficulty_at_exit.squeeze()
            print(difficulty_at_exit)
    
            if subj_df.loc[entry_row_idx, 'Action']=='LEAVE':
                num_harvests = 0
            else:
                unique_harvest_reqs = np.unique(subj_df.loc[entry_row_idx:exit_row_idx, 'RequiredPresses'])
                # print(unique_harvest_reqs)
                num_harvests = len(unique_harvest_reqs)
    
                for har in unique_harvest_reqs:
                    times_idxs = subj_df[(subj_df['TreeNumber']==tree) & (subj_df['RequiredPresses']==har)].index
                    time_dur_har = subj_df['TimeStamp'][times_idxs[-1]] - subj_df['TimeStamp'][times_idxs[0]]
                    time_dur_har = int(time_dur_har)
                    dur_per_harvest.append(time_dur_har)
                    num_presses = subj_df['RequiredPresses'][times_idxs[0]]
                    num_presses_arr.append(int(num_presses))
                    scores_harvest.append(int(subj_df['score'][times_idx[0]]))
    
            score_at_exit = subj_df.loc[exit_row_idx, 'score']
            score_tree = score_at_exit - prev_score
            prev_score = score_at_exit
            print(f"score_tree: {score_tree}")
            har_dur_list = [subj, block, tree, tree_type, har_dur, num_harvests, difficulty_at_exit, score_tree, 
                            dur_per_harvest, num_presses_arr]
            print(har_dur_list)
            har_dur_df_list.append(har_dur_list)
    
    har_dur_df = pd.DataFrame(har_dur_df_list, columns = headers)

    if shortFirst:
        out_filename = "har_dur_shortfirst.csv"
    else:
        out_filename = "har_dur_longfirst.csv"
        
    har_dur_df.to_csv(out_filename, index=False, header=True)


In [ ]:
create_condensed_df(data_sf, shortFirst = True)

In [ ]:
create_condensed_df(data_lf, shortFirst = False)